In [1]:
%pylab inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('data/Train.csv')

Populating the interactive namespace from numpy and matplotlib


In [6]:
train[['ModelID','saledate','state','MachineHoursCurrentMeter']].head()

,ModelID,saledate,state,MachineHoursCurrentMeter
0,3157,11/16/2006 0:00,Alabama,68.0
1,77,3/26/2004 0:00,North Carolina,4640.0
2,7009,2/26/2004 0:00,New York,2838.0
3,332,5/19/2011 0:00,Texas,3486.0
4,17311,7/23/2009 0:00,New York,722.0


In [5]:
train['saleyear']=[datetime.strptime(row, '%m/%d/%Y %H:%M') for row in train['saledate']]
train['saleyear'] = train['saleyear'].dt.year
train = train[['ModelID', 'saleyear', 'SalePrice']]
train_sorted = train.sort_values(['ModelID','saleyear'])
train_sorted_collapsed = train_sorted.groupby(['ModelID','saleyear']).median().reset_index()

In [6]:
d = {}
for i in range(len(train_sorted_collapsed)):
    if train_sorted_collapsed['ModelID'][i] in d:
        d[train_sorted_collapsed['ModelID'][i]].append((train_sorted_collapsed['saleyear'][i],train_sorted_collapsed['SalePrice'][i]))
    else:
        d[train_sorted_collapsed['ModelID'][i]] = [(train_sorted_collapsed['saleyear'][i],train_sorted_collapsed['SalePrice'][i])]

In [7]:
test = pd.read_csv('data/Test.csv')
test['saleyear']=[datetime.strptime(row, '%m/%d/%Y %H:%M') for row in test['saledate']]
test['saleyear'] = test['saleyear'].dt.year
test = test[['SalesID', 'ModelID', 'saleyear']]
test['SalePrice'] = 0

In [8]:
for i in range(len(test)):
    if test.iloc[i][1] not in d: 
        test['SalePrice'][i] = 24000
    else:
        cyear, cprice, yeardiff = 2000, 0, 50
        for j in d[test.iloc[i][1]]:
            if np.abs(test.iloc[i][2]-j[0]) < cyear:
                cyear, cprice, yeardiff = j[0], j[1], test.iloc[i][2]-j[0]
        test['SalePrice'][i] = cprice * np.exp(0.04*yeardiff)

In [9]:
ids = pd.Series(test['SalesID'])
price_predict = pd.Series(test['SalePrice'])

In [134]:
pd.concat([ids, price_predict], axis=1).to_csv('data/predictions.csv', index=False, header=['SalesID','SalePrice'])